In [239]:
import glob
import os.path
from os import path
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D, InputLayer, Dense, UpSampling3D, Conv3D, AveragePooling2D
from tensorflow.keras import layers
from keras.models import Model
from keras.optimizers import SGD
from skimage.color import lab2rgb, rgb2lab

In [240]:
images_glob = glob.glob('/home/mario/MIU/ML/final-project/image-colorizer-notebook/unlabeled2017_subsample/*.jpg')
images_glob = images_glob[:2800]

lab_images = np.array([np.array(load_img(img_path)) / 255 for img_path in images_glob])
done = "done loading data"
done
lab_images.shape

MemoryError: Unable to allocate 5.63 GiB for an array with shape (2800, 300, 300, 3) and data type float64

In [ ]:
def normalize_lab(img):
  return (img + [0, 128, 128])/[100, 255, 255]

def denormalize_lab(img):
  return (img * [100, 255, 255]) - [0, 128, 128]

def convert_rgb_to_lab(img):
  return normalize_lab(rgb2lab(img))

def convert_lab_to_rgb(img):
  return lab2rgb(denormalize_lab(img))

def extract_grayscalergb_from_lab(image):
  bw = np.zeros(image.shape)

In [ ]:
#actually convert to LAB and scale according to L*a*b format values range
lab_images = np.array([convert_rgb_to_lab(img) for img in lab_images])

print(lab_images.shape)

done = 'done converting to grayscale'
done

In [ ]:
plt.figure(1)
plt.imshow(lab_images[666])
plt.figure(2)
plt.imshow(convert_lab_to_rgb(lab_images[666]), cmap='gray')
plt.show()

In [ ]:
IMG_WIDTH = 300
IMG_HEIGHT = 300
# We will predict the a*b layers from the L layer of LAB formatted image
inputs = np.array(lab_images[:, :, :, 0]).reshape(len(lab_images), IMG_WIDTH, IMG_HEIGHT, 1)
outputs = np.array(lab_images[:, :, :, 1:]).reshape(len(lab_images), IMG_WIDTH, IMG_HEIGHT, 2)

# split is 70 * 30 for training. Set to 95 * 5 for actual testing
train_X, test_X, train_y, test_y = train_test_split(inputs, outputs, train_size=0.90)

test_X, test_y = np.array(test_X), np.array(test_y)

done = 'done splitting data'
done

In [ ]:
resnet = ResNet50(weights='imagenet', include_top=False)

In [ ]:
# Create working prediction model

# TODO: add imagenet layer and freeze
# TODO: add CAM by getting Conv2D in the middle, summing them up elementwise and plotting over original input.

if not path.exists("/home/mario/MIU/ML/final-project/image-colorizer-notebook/saved.model"):
    model = Sequential()
    model.add(InputLayer(input_shape=(IMG_WIDTH, IMG_HEIGHT, 1)))
    model.add(UpSampling3D((1, 1, 3)))
    model.add(resnet)
    model.layers[-1].trainable = False
    model.add(UpSampling2D((4, 4)))
    model.add(Conv2D(8, (4, 4), activation='relu', padding='valid', strides=2))
    model.add(Conv2D(16, (4, 4), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=1))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((3, 3)))
    model.add(Conv2D(16, (2, 2), activation='relu', padding='same'))
    model.add(UpSampling2D((5, 5)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
    model.summary()
print('done creating layer stack')

In [ ]:
if path.exists("/home/mario/MIU/ML/final-project/image-colorizer-notebook/saved.model"):
  model = keras.models.load_model("/home/mario/MIU/ML/final-project/image-colorizer-notebook/saved.model")
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
print('done compiling...')
model.fit(x=train_X, y=train_y, batch_size=50, epochs=100, verbose=2)
model.save("/home/mario/MIU/ML/final-project/image-colorizer-notebook/saved.model")

In [ ]:
# Warning: takes long to load
model.evaluate(test_X, test_y)

In [ ]:
test_pred[0]

In [ ]:
visual_test_idx = 997

test_pred = model.predict(np.array([inputs[visual_test_idx]]))

predicted_LAB = np.zeros((IMG_WIDTH, IMG_HEIGHT, 3))
predicted_LAB[:, :, 0] = inputs[visual_test_idx][:, :, 0]
predicted_LAB[:, :, 1:] = test_pred[0]
predicted_RGB = convert_lab_to_rgb(predicted_LAB)

plt.figure(4)
plt.imshow(predicted_RGB)
plt.show()